In [ ]:
import pandas as pd
import finterstellar as fs

데이터 로딩

In [ ]:
coin = fs.CoinPrice()

In [ ]:
cd = 'BTC'

In [ ]:
df = coin.bithumb_historical_price(cd)
df.head()

Moving average 구하기

In [ ]:
short = 10    # 단기 기준
mid = 60    # 중기 기준

In [ ]:
ma = pd.DataFrame()
ma[cd] = df['close']
ma['short'] = df['close'].rolling(short).mean()   # 데이터프레임[데이터].rolling(기준일자).mean() - 이동평균 계산
ma['mid'] = df['close'].rolling(mid).mean()
ma.tail(10)

In [ ]:
base_date = df.index[0]   # 기준점 설정
ma = ma[base_date:].copy()    # 기준일 이후 데이터만 가져와 ma에 저장
ma.head()

그래프 그리기 (fs 라이브러리 이용)

In [ ]:
v = fs.VisualizeIntraday()      # fs 라이브러리의 Visualize() 모듈을 불러와 v로 지정

In [ ]:
v.price_view(ma, base_date, cd, (15, 2), make_file=False)
# price_view(데이터프레임, 기준일자, [비교대상(복수종목)], (챠트사이즈)) 함수 - 그래프 그리기

In [ ]:
v.price_view(ma, base_date, ['mid', 'short'], (15, 2))
# price_view(데이터프레임, 기준일자, [비교대상(복수종목)], (챠트사이즈)) 함수 - 그래프 그리기

# MA로 트레이딩을 한다면?

short과 mid가 교차할 때 매매하는 로직을 세우기 위해

In [ ]:
ma['s-m'] = ma['short'] - ma['mid']
ma.tail(3)
# 그렇다면, s-m 값이 +라면 상승추세인걸로

In [ ]:
def trend_tradings(sample, book, cd, factor):
    for i in sample.index:    # 시세를 하나씩 읽어가며
        if sample.loc[i, factor] >= 0:   # 상승추세라면
            book.loc[i, 't '+cd] = 'buy'    # buy해라
        elif sample.loc[i, factor] < 0:    # 하락추세라면
            book.loc[i, 't '+cd] = ''    # 털어라
    return (book) 

백테스팅을 위한 트레이드 북 생성

In [ ]:
trd = fs.Trade()   # fs 라이브러리의 Trade() 모듈을 불러와 trd로 지정

In [ ]:
book = trd.create_trade_book(ma, [cd])    
# trd 모듈의 create_trade_book(데이터프레임, [종목코드]) 함수를 이용해 트레이딩북 생성
book.tail()

트렌드 트레이딩 전략 실행

In [ ]:
book = trend_tradings(ma, book, cd, 's-m')
book.tail()

포지션 산출

In [ ]:
book = trd.position(book, [cd])
# position(트레이딩북, 종목코드) - 매매전략에 따른 포지션을 판단해 트레이딩북에 기록
book.tail()

손익 계산

In [ ]:
fund_rtn = trd.returns(book, cd, display=True)
# returns(트레이딩북, 종목코드) - 백테스팅 수익률 계산

벤치마크 수익률 계산

In [ ]:
bm_rtn = trd.benchmark_return(book, [cd])
# benchmark_return(트레이딩북, 종목코드) - 벤치마크 수익률(전략 미이용시) 계산

In [ ]:
exs_rtn = trd.excess_return(fund_rtn, bm_rtn)
# excess_return(전략수익률, 벤치마크수익률) - 전략의 벤치마크 대비 초과수익률 계산